In [1]:
from factor_analyzer import FactorAnalyzer
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import math

data = pd.read_csv('인사이콘3_Data.csv')
data.head()

,RESPID,Q1.1,Q1.2,Q1.3,Q1.4,Q1.5,Q1.6,Q1.7,Q1.8,Q1.9,...,Q16.20,Q16.21,Q16.22,Q16.23,Q16.24,Q16.25,Q16.26,Q16.27,Q17,Q18
0,23822,5,1,5,5,5,5,5,3,5,...,4,4,4,4,5,4,4,4,0,1
1,224795,3,2,4,4,4,4,5,3,4,...,4,3,3,4,3,3,3,3,1,1
2,20952,3,1,4,4,5,4,4,2,4,...,5,3,3,5,2,4,3,5,1,0
3,5612,5,2,3,4,5,5,3,3,3,...,4,4,4,4,2,5,5,4,1,0
4,15016,4,2,4,5,5,4,5,4,5,...,4,4,5,5,3,4,5,4,0,0


### data를 1차원으로 차원축소 (방법은 그냥 평균내기)

In [2]:
data['mean'] = data.iloc[:,1:].mean(axis=1)
data.head()

,RESPID,Q1.1,Q1.2,Q1.3,Q1.4,Q1.5,Q1.6,Q1.7,Q1.8,Q1.9,...,Q16.21,Q16.22,Q16.23,Q16.24,Q16.25,Q16.26,Q16.27,Q17,Q18,mean
0,23822,5,1,5,5,5,5,5,3,5,...,4,4,4,5,4,4,4,0,1,6.617284
1,224795,3,2,4,4,4,4,5,3,4,...,3,3,4,3,3,3,3,1,1,4.567901
2,20952,3,1,4,4,5,4,4,2,4,...,3,3,5,2,4,3,5,1,0,4.839506
3,5612,5,2,3,4,5,5,3,3,3,...,4,4,4,2,5,5,4,1,0,4.876543
4,15016,4,2,4,5,5,4,5,4,5,...,4,5,5,3,4,5,4,0,0,4.703704


### 축소된 data를 median값으로 2분할 ( median보다 크면 1 작으면 0 ) - 여기서 좋은 꼼수가 필요.

In [3]:
m = data['mean'].median()
data['mean'] = data['mean'].apply(lambda x: 1 if x>m else 0)
data['mean'].head()

0    1
1    1
2    1
3    1
4    1
Name: mean, dtype: int64

### 해당 값을 model에 적용

#### silhouette score는 당연히 1.0

In [14]:
kmeans = KMeans(n_clusters=2, random_state=1422)
kmeans.fit(data[['mean']])
clusters = kmeans.predict(data[['mean']])
score = silhouette_score(data[['mean']], clusters)
score

1.0

#### 표준편차점수 계산하기

- 각 군집별 개수를 통해 구합니당

In [8]:
nums = pd.Series(clusters).value_counts()
nums

0    1421
1    1392
dtype: int64

- 엄IU의 수식에 따라 구해봅니다.

In [52]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [53]:
print(f"표준편차는           {np.std(nums)}")
print(f"거기에 로그 씌운거는 {np.log(np.std(nums))}")
print(f"최종 표준편차 점수는 {sigmoid(np.log(np.std(nums)))}")

표준편차는           0.372677996249965
거기에 로그 씌운거는 -0.9870405130110047
최종 표준편차 점수는 0.27149702790318514


#### 최종 점수

In [54]:
print(f'최종 점수는 {score-sigmoid(np.log(np.std(nums)))}')

최종 점수는 0.7285029720968148
